In [2]:
import json
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import numpy as np
from tensorflow.keras import models, layers, optimizers, losses, metrics
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
import csv
import pandas as pd
from tensorflow.keras.layers import Dense, Embedding, Flatten
from tensorflow.keras import Input, Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [3]:
x, y = [], []
file1 = "./comment-labeling.csv"
file2 = "./naver-ratings.csv"

In [6]:
f = open(file1, 'r', encoding='utf-8')
read = csv.reader(f)
for line in read:
    emotion = float(line[-1])
    x.append(line[0])
    y.append(emotion)
    
f.close()

In [7]:
x2, y2 = [], []
f = open(file2, 'r', encoding='utf-8')
read = csv.reader(f)
for line in read:
    emotion = float(line[-1])
    y2.append(emotion)
    x2.append(line[0])
    
f.close()

In [8]:
test_percent = 0.2

In [9]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_percent)
a, b, c, d = train_test_split(x2, y2, test_size=test_percent)

In [10]:
x_train += a
x_test += b
y_train += c
y_test += d

In [17]:
def build_model(train_data):
    train_data = tf.data.Dataset.from_tensor_slices(train_data)
    model = Sequential()
    model.add(Input(shape=(1,), dtype="string"))
    max_tokens = 15000
    max_len = 50
    vectorize_layer = TextVectorization(
        max_tokens=max_tokens,
        output_mode="int",
        output_sequence_length=max_len
    )
    
    vectorize_layer.adapt(train_data.batch(64))
    model.add(vectorize_layer)
    model.add(layers.Embedding(max_tokens + 1, output_dim=200))
    model.add(Flatten())
    model.add(Dense(8, activation="relu"))
    model.add(Dense(1, activation="sigmoid"))
    return model

In [34]:
tf.config.experimental_enable_xla=True

In [35]:
rnn_model = build_model(x_train)
rnn_model.compile(
    optimizer="adam",
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [36]:
history = rnn_model.fit(
    x_train,
    y_train,
    epochs=5,
    batch_size=128,
    validation_data=(x_test, y_test)
)

Epoch 1/5
   1/1269 [..............................] - ETA: 3:02 - loss: 0.6933 - accuracy: 0.5000

C:\Users\kwarkmc\.conda\envs\capstone\lib\site-packages\tensorflow\python\data\ops\structured_function.py:256: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


1269/1269 [==============================] - 94s 74ms/step - loss: 0.6673 - accuracy: 0.5344 - val_loss: 0.4096 - val_accuracy: 0.8094
Epoch 2/5
1269/1269 [==============================] - 95s 75ms/step - loss: 0.3541 - accuracy: 0.8422 - val_loss: 0.3497 - val_accuracy: 0.8433
Epoch 3/5
1269/1269 [==============================] - 94s 74ms/step - loss: 0.2468 - accuracy: 0.8939 - val_loss: 0.3909 - val_accuracy: 0.8332
Epoch 4/5
1269/1269 [==============================] - 94s 74ms/step - loss: 0.1675 - accuracy: 0.9295 - val_loss: 0.5141 - val_accuracy: 0.8235
Epoch 5/5
1269/1269 [==============================] - 94s 74ms/step - loss: 0.1264 - accuracy: 0.9459 - val_loss: 0.6455 - val_accuracy: 0.8181


In [37]:
rnn_model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_1 (TextV  (None, 50)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 50, 200)           3000200   
                                                                 
 flatten_1 (Flatten)         (None, 10000)             0         
                                                                 
 dense_2 (Dense)             (None, 8)                 80008     
                                                                 
 dense_3 (Dense)             (None, 1)                 9         
                                                                 
Total params: 3,080,217
Trainable params: 3,080,217
Non-trainable params: 0
____________________________________________

In [38]:
rnn_model.save("WtoE", save_format='tf')

INFO:tensorflow:Assets written to: WtoE\assets


INFO:tensorflow:Assets written to: WtoE\assets
